# Project - Deeplabv3

###  1.Create data dictionary by train, valid and test

In [1]:
import pandas as pd

In [2]:
# divide data from test valid train
meta_split = pd.read_csv(f'/root/autodl-tmp/deeplabv3/archive/turtles-data/data/metadata_splits.csv')
# get main data from csv
meta_data = meta_split[['id', 'file_name', 'split_open']]
# get types name
data_type = set(meta_data['split_open'])

# get each type set and their id and file names
meta_data_dict = {}
for type in data_type:
    meta_data_dict[type] = []
for index, row in meta_data.iterrows():
    meta_data_dict[row['split_open']].append((row['id'], row['file_name']))

print(f"train set number: {len(meta_data_dict['train'])}")
print(f"valid set number: {len(meta_data_dict['valid'])}")
print(f"test set number: {len(meta_data_dict['test'])}")
meta_data

train set number: 5303
valid set number: 1118
test set number: 2308


,id,file_name,split_open
0,1,images/t001/CAluWEgwPX.JPG,train
1,2,images/t001/EKyrFKHQzh.JPG,train
2,3,images/t001/ELAvEqeXxT.JPG,train
3,4,images/t001/IxRLFwTGCv.JPG,train
4,5,images/t001/LKCJAhfLBJ.JPG,train
...,...,...,...
8724,8725,images/t610/miUGGSioXO.jpeg,test
8725,8726,images/t610/aOzTdMCkzF.jpeg,test
8726,8727,images/t610/ZmTLXySHIZ.jpeg,test
8727,8728,images/t610/qVDYBLbzda.jpeg,test


### 2.Set class dataset by using coco

for getting image and mask, then we can put into model

In [3]:
import json
from pycocotools.coco import COCO
from pprint import pprint
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
from torch.utils.data import Dataset 
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torchvision.models.segmentation
import torch.optim as optim 
import torch

In [4]:
# set img dir
image_dir = '/root/autodl-tmp/deeplabv3/archive/turtles-data/data/'       

# json to coco
coco = COCO('/root/autodl-tmp/deeplabv3/archive/turtles-data/data/annotations.json')

# image and mask transform method
transform_method = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor()
])

target_transform_method = transforms.Compose([
    transforms.Resize((512, 512), interpolation=Image.NEAREST), 
    transforms.Lambda(lambda mask: torch.from_numpy(np.array(mask)).long())
])

loading annotations into memory...
Done (t=4.17s)
creating index...
index created!


In [5]:
class CustomDataset(Dataset):
    def __init__(self, image_dir, coco, meta_data_dict, data_type, transform=None, target_transform=None):
        self.image_dir = image_dir  # original image dir
        self.coco = coco # coco
        self.data_type = data_type # train valid test
        self.transform = transform  # transform method
        self.target_transform = target_transform  # transform method
        self.images = [] # all original images path in train set

        data_list = meta_data_dict[self.data_type]
        img_Ids = self.coco.getImgIds()

        for i in img_Ids:
            # if img_id in data_type list is true
            if any(t[0] == i for t in data_list):
                self.images.append(self.coco.loadImgs(i)[0])

    def process_mask(self, image_id, image):
        category_map = {
            1: 1,  # turtles
            2: 2,  # flipper
            3: 3   # head
        }
        
        mask = np.zeros((image.size[1], image.size[0]), dtype=np.uint8)
        for cat_id, target_value in category_map.items():
            ann_ids = self.coco.getAnnIds(imgIds=image_id, catIds=cat_id, iscrowd=None)
            anns = self.coco.loadAnns(ann_ids)
            for ann in anns:
                ann_mask = self.coco.annToMask(ann)
                mask[ann_mask > 0] = target_value  
                
        return mask

    def __len__(self):  # set dataset size
        return len(self.images)

    def __getitem__(self, idx):
        img = self.images[idx]
        # get img id
        img_id = img['id']
        # get img path
        img_file_name = img['file_name']

        image = Image.open(f"{self.image_dir}/"+img_file_name).convert("RGB") # transfor to PIL
        mask = self.process_mask(img_id, image)
        mask = Image.fromarray(mask.astype('uint8'))  # transfor to PIL

        # transform image and mask to [C, H, W]
        image = self.transform(image) 
        mask = self.target_transform(mask)

        return image, mask 
    
    # image: PIL [H,W,C] -> [C,H,W]
    # mask: numpy -> PIL [H,W,C] -> [C,H,W]


### 3.Create dataset for train, valid, test

In [6]:
# create train dateset
train_dataset = CustomDataset(
    image_dir=image_dir,
    coco=coco,
    transform=transform_method,
    meta_data_dict=meta_data_dict,
    target_transform=target_transform_method,
    data_type='train'
)

# create valid dataset
valid_dataset = CustomDataset(
    image_dir=image_dir,
    coco=coco,
    transform=transform_method,
    meta_data_dict=meta_data_dict,
    target_transform=target_transform_method,
    data_type='valid'
)

# create test dataset
test_dataset = CustomDataset(
    image_dir=image_dir,
    coco=coco,
    transform=transform_method,
    meta_data_dict=meta_data_dict,
    target_transform=target_transform_method,
    data_type='test'
)

### 4.Create dataloader
divide dataset into samll dataset in order to improving effiency

In [7]:
# create valid DataLoader
valid_dataloader = DataLoader(valid_dataset, batch_size=8, shuffle=True)
# create test DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=True)
# create train DataLoader
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# see the data set size
for images, masks in train_dataloader:
    print("Batch of images:", images.shape)  # show the first image tensor
    print("Batch of masks:", masks.shape)    # show the first mask tensor
    unique_values = torch.unique(masks)     
    print("Masks dtype:", masks.dtype)      # show the first mask dtype
    print("Masks value:", unique_values)    # show the first mask uniqe
    break  # just want to the first one


Batch of images: torch.Size([16, 3, 512, 512])
Batch of masks: torch.Size([16, 512, 512])
Masks dtype: torch.int64
Masks value: tensor([0, 1, 2, 3])


### 5.Show dataloader size and dataset size

In [8]:
print("Total number of images in the dataset:", len(train_dataset))
print("Total number of images in the dataset:", len(valid_dataset))
print("Total number of images in the dataset:", len(test_dataset))
print("Total number of images in the dataset:", len(train_dataloader))

Total number of images in the dataset: 5303
Total number of images in the dataset: 1118
Total number of images in the dataset: 2308
Total number of images in the dataset: 332


### 6.Using deeplabv3 model to get iou and miou

In [9]:
num_classes = 4  # set classes number

# using deeplabv3
model = torchvision.models.segmentation.deeplabv3_resnet50(pretrained=False)

# Modify the category header of the model to fit the number of custom categories
model.classifier[4] = torch.nn.Conv2d(256, num_classes, kernel_size=(1,1))

# using cuba gpu
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# loss function
criterion = torch.nn.CrossEntropyLoss()

# Define the optimizer (Adam optimizer, learning rate 0.0001)
optimizer = optim.Adam(model.parameters(), lr=0.0001)


/root/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [10]:
# caculate iou function
def calculate_iou(pred_mask, true_mask, num_classes):
    ious = []
    pred_mask = pred_mask.view(-1)  
    true_mask = true_mask.view(-1)  

    for cls in range(1, num_classes):  # only caculate iou for three classes
        pred_inds = (pred_mask == cls)
        true_inds = (true_mask == cls)
        
        intersection = (pred_inds & true_inds).sum().float()  
        union = (pred_inds | true_inds).sum().float()  

        if union == 0:
            ious.append(float('nan'))  # avoid denominators of zero
        else:
            ious.append((intersection / union).item())  # append iou in list

    return ious # return list

# ----------------------
# Training Phase
# ----------------------
num_epochs = 25 # set number of epochs

for epoch in range(num_epochs):
    model.train()  # train model
    running_loss = 0.0

    for images, masks in train_dataloader:
        images, masks = images.to(device), masks.to(device)
        masks = masks.squeeze(1)  

        optimizer.zero_grad()  
        outputs = model(images)['out']  
        loss = criterion(outputs, masks.long())  
        loss.backward()  
        optimizer.step()  

        running_loss += loss.item()  # caculate loss

    # print loss
    avg_train_loss = running_loss / len(train_dataloader)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Training Loss: {avg_train_loss:.4f}")

    # ----------------------
    # Validation Phase
    # ----------------------
    model.eval()  # eval model
    all_ious = []  # record all iou in one epoch

    with torch.no_grad():  # no grad
        for images, masks in valid_dataloader:
            images, masks = images.to(device), masks.to(device)
            masks = masks.squeeze(1)

            outputs = model(images)['out']
            pred_masks = torch.argmax(outputs, dim=1)

            for i in range(images.size(0)):
                ious = calculate_iou(pred_masks[i], masks[i], num_classes=num_classes)
                all_ious.append(ious)

    # caculate miou and average of total miou
    all_ious_tensor = torch.tensor(all_ious)
    mean_ious = torch.nanmean(all_ious_tensor, dim=0) 
    avg_all_mious = torch.nanmean(mean_ious).item()

    # print results
    print(f"Epoch [{epoch + 1}/{num_epochs}], Validation the average of three class miou: {avg_all_mious:.4f}")
    for cls in range(num_classes):
        if cls == 0:
            print(f" - Class {cls + 1} 'turtle' mIoU: {mean_ious[cls].item():.4f}")
        elif cls == 1:
            print(f" - Class {cls + 1} 'flipper' mIoU: {mean_ious[cls].item():.4f}")
        elif cls == 2:
            print(f" - Class {cls + 1} 'head' mIoU: {mean_ious[cls].item():.4f}")
    
    print("----------------------------------------------------------------")

# ----------------------
# Testing Phase
# ----------------------
model.eval()
test_ious = []

with torch.no_grad():
    for images, masks in test_dataloader:
        images, masks = images.to(device), masks.to(device)
        masks = masks.squeeze(1)
        
        # [betch size, number of classes, height, width] -> [betch size, number of classes, height, width] 
        outputs = model(images)['out'] 
        # [betch size, number of classes, height, width] -> [betch size, height, width] 
        pred_masks = torch.argmax(outputs, dim=1)
        
         # image.size = [betch size, number of classes, height, width]
         # image.size(0) = betch size = 16
         # for i in range(16)
        for i in range(images.size(0)):
            ious = calculate_iou(pred_masks[i], masks[i], num_classes=num_classes)
            test_ious.append(ious)

# caculate miou and average of total miou
all_ious_tensor = torch.tensor(test_ious)
mean_ious = torch.nanmean(all_ious_tensor, dim=0)
avg_all_mious = torch.nanmean(mean_ious).item()

# print results
print(f"Testing the average of three class miou: {avg_all_mious:.4f}")
for cls in range(num_classes):
    if cls == 0:
        print(f" - Class {cls + 1} 'turtle' mIoU: {mean_ious[cls].item():.4f}")
    elif cls == 1:
        print(f" - Class {cls + 1} 'flipper' mIoU: {mean_ious[cls].item():.4f}")
    elif cls == 2:
        print(f" - Class {cls + 1} 'head' mIoU: {mean_ious[cls].item():.4f}")
        
        
    #images [betch size, number of classes, height, width] 
    #masks [betch size, height, width]


Epoch [1/25], Training Loss: 0.1506
Epoch [1/25], Validation the average of three class miou: 0.7814
 - Class 1 'turtle' mIoU: 0.8714
 - Class 2 'flipper' mIoU: 0.7221
 - Class 3 'head' mIoU: 0.7508
----------------------------------------------------------------
Epoch [2/25], Training Loss: 0.0384
Epoch [2/25], Validation the average of three class miou: 0.8166
 - Class 1 'turtle' mIoU: 0.8943
 - Class 2 'flipper' mIoU: 0.7716
 - Class 3 'head' mIoU: 0.7841
----------------------------------------------------------------
Epoch [3/25], Training Loss: 0.0232
Epoch [3/25], Validation the average of three class miou: 0.8205
 - Class 1 'turtle' mIoU: 0.9047
 - Class 2 'flipper' mIoU: 0.7620
 - Class 3 'head' mIoU: 0.7948
----------------------------------------------------------------
Epoch [4/25], Training Loss: 0.0187
Epoch [4/25], Validation the average of three class miou: 0.8000
 - Class 1 'turtle' mIoU: 0.8761
 - Class 2 'flipper' mIoU: 0.7694
 - Class 3 'head' mIoU: 0.7545
---------

In [1]:
for i in range(1, 4):
    print(i)

1
2
3
